# Lower Mantle Composition using BurnMan


Global geophysicists have an ongoing discussion on the degree of whole-mantle convection versus the degree of two-layered convection. There is good certainty that in the present day there is exchange of material between the upper and lower mantle in the form of downgoing slabs and upgoing mantle plumes. The question remains if this has always been the case and if this exchange has been sufficient to create a well-mixed mantle. One way to assess this is to test if the upper and lower mantles have similar compositions. For the upper mantle we have more direct samples from volcanoes. For the lower mantle we can attempt to determine the composition based on seismic velocity models. 

[BurnMan](https://geodynamics.github.io/burnman/) is an open source Python toolkit that computes the thermo-elastic properties using an equation-of-state for composites at high pressures and temperatures. The equation-of-state parameters come from databases compiled from many experiments of minerals under high-(P,T) conditions. The code provides the tools to directly compare the computed velocities to seismically observed ones.
To read more background on BurnMan and the fitting of lower mantle compositions, see:

Cottaar, S, T Heister, I Rose, and C Unterborn. 2014. “BurnMan–a Lower Mantle Mineral Physics Toolkit.” Geochem. Geophys. Geosyst. 15 (4): 1164–79.



### Step 0. Import modules

Import of relevant modules. Only need to run this step once.

In [ ]:
%matplotlib inline
import os, sys, numpy as np, matplotlib.pyplot as plt # General libraries
from matplotlib import cm  # Library used for colormap
from collections import OrderedDict

# Import BurnMan to compute velocities for a given composition
if '../burnman_v1.0.1' not in sys.path:
    sys.path.insert(0,'../burnman_v1.0.1')
import burnman
from burnman import minerals

### Step 1. Exploring 1D models

The code below plots the lower mantle Vs, Vp and density for three different 1D models. 

a.	What are these models based on?


b.	Why might 1D Earth models not be a good reflection of the composition in the lower mantle?



In [ ]:
# List of seismic 1D models
seismic_models = [
        burnman.seismic.PREM(),
        burnman.seismic.STW105(),
        burnman.seismic.AK135()]
colors = ['r', 'b', 'm', 'k']

# initiate figure and subplots
fig = plt.figure(figsize=(14, 5))
ax = [fig.add_subplot(1, 3, i) for i in range(1, 4)]
    
# Run through models and variables
for m in range(len(seismic_models)):
        # get depths at which the model is defined
        depths_seis = seismic_models[m].internal_depth_list(
            mindepth=0, maxdepth=6371e3)
        Vp, Vs, rho = seismic_models[m].evaluate(
            ['v_p', 'v_s', 'density'], depths_seis)

        # PLOTTING the results. While all parameters from BurnMan are given in
        # SI units (e.g. m/s for velocity), they are here converted to other
        # units for plotting purposes.
        ax[0].plot(depths_seis / 1.e3, Vp / 1.e3, color=colors[m], linestyle='-')
        ax[1].plot(depths_seis / 1.e3, Vs / 1.e3, color=colors[m], linestyle='-')
        ax[2].plot(
            depths_seis / 1.e3,
            rho / 1.e3,
            color=colors[m],
            linestyle='-',
            label=seismic_models[m].__class__.__name__)

# Beautify plots. The values are bounded to only show the lower mantle,
# but feel free to play around with this to see the rest of the planet.
ax[0].set_xlabel('depth in km')
ax[0].set_ylabel('Vp in km/s')
ax[0].set_xlim([750., 2700.])
ax[0].set_ylim([10.5, 14.])
ax[1].set_xlabel('depth in km')
ax[1].set_ylabel('Vs in km/s')
ax[1].set_xlim([750., 2700.])
ax[1].set_ylim([6., 7.5])
ax[2].set_xlabel('depth in km')
ax[2].set_ylabel('density in kg/m^3 ')
ax[2].set_xlim([750., 2700.])
ax[2].set_ylim([4., 5.5])
ax[2].legend(loc=2, borderaxespad=0.)


### Step 2. Fitting 1D models

We will now test two different compositional models for the lower mantle. The pyrolitic model represents a composition similar to the upper mantle (ignoring Al and other minor components):

-	Pyrolitic model
    - 75% perovskite or bridgemanite (Mg, Fe)SiO3 with 94% Mg and 6% Fe
    - 18% ferropericlase (Mg,Fe)O with 80% Mg and 20%Fe
    - 7% Ca-perovskite CaSiO3

The chondritic model is a competing model that suggests the composition the lower mantle should have if composition of the bulk Earth is represented by chondritic meteorites. In this case the lower mantle is enriched in Si relative to the upper mantle.

-	Chondritic model
    - 88% perovskite or bridgemanite (Mg, Fe)SiO3 with 94% Mg and 6% Fe
    - 5% ferropericlase (Mg,Fe)O with 80% Mg and 20%Fe
    - 7% Ca-perovskite CaSiO3

To model these compositions, we are using the mineral database published in Stixrude and Lithgow-Bertelloni (2011). 

Run through the scripts as part of step 2 below. 

c.	Which model best fits the seismically observed velocities and density best? 

d.	Uncomment the plotting commands at the end of 2.7. Now a rock called 'your_mantle' is also plotted. It is currently set to a pyrolitic composition, but you can change this rock a couple code cells above. Vary the ratio of perovskite to periclase, or the Mg/Fe ratio in the minerals.  You can also change the temperature at the top of the mantle. NOTE: with each change you have to rerun the cell blocks (also the block that recomputes the velocities). With what composition can you improve the fit to radial velocities and density? You will find it is challenging to fit all three at once. Don't worry if you do not find a perfect solution!


#### 2.1 Define depth and pressure arrays

In [ ]:
# 20 points within the lower mantle
depths = np.linspace(750e3, 2700e3, 20)
# convert depth to pressure using PREM
[pressures] = seismic_models[0].evaluate(['pressure'], depths)

##### 2.2 Predefine solid solutions and mineral used

In [ ]:
# Perovksite solid solution
frac_mg = 0.94
frac_fe = 0.06
frac_al = 0.00
mg_fe_perovskite = minerals.SLB_2011.mg_fe_perovskite()
mg_fe_perovskite.set_composition([frac_mg, frac_fe, frac_al])

# Ferropericlase solid solution
frac_mg = 0.8
frac_fe = 0.2
mg_fe_periclase = minerals.SLB_2011.ferropericlase()
mg_fe_periclase.set_composition([frac_mg,frac_fe])
    
# Ca Perovskite
ca_perovskite = minerals.SLB_2011.ca_perovskite()

##### 2.3 Build pyrolitic and chondritic compositions

In [ ]:
# Pyrolitic composition
pyr_pv = 0.75
pyr_fp = 0.18
pyr_capv = 0.07
pyrolitic_mantle = burnman.Composite(
        [mg_fe_perovskite, mg_fe_periclase, ca_perovskite], [pyr_pv, pyr_fp, pyr_capv], name = 'Pyrolitic')


# Chondritic composition
chon_pv = 0.88
chon_fp = 0.05
chon_capv = 0.07
chondritic_mantle = burnman.Composite(
        [mg_fe_perovskite, mg_fe_periclase, ca_perovskite], [chon_pv, chon_fp, chon_capv], name = 'Chondritic')
 

##### 2.4 Build your own composition
You will modify this later.

In [ ]:
# Perovksite solid solution, here you can set the iron and aluminium content
frac_mg = 0.94
frac_fe = 0.06
frac_al = 0.00
mg_fe_perovskite = minerals.SLB_2011.mg_fe_perovskite()
mg_fe_perovskite.set_composition([frac_mg, frac_fe, frac_al])
    
# Ferropericlase solid solution, here you can set the iron content
frac_mg = 0.8
frac_fe = 0.2
mg_fe_periclase = minerals.SLB_2011.ferropericlase()
mg_fe_periclase.set_composition([frac_mg,frac_fe])
    
# Ca Perovskite
ca_perovskite = minerals.SLB_2011.ca_perovskite()

# Here you can set the relative ratios
pyr_pv = 0.75
pyr_fp = 0.18
pyr_capv = 0.07
your_mantle = burnman.Composite([mg_fe_perovskite, mg_fe_periclase, ca_perovskite], [pyr_pv, pyr_fp, pyr_capv], name = 'Your Mantle')

##### 2.5 Compute adiabatic temperatures

In [ ]:
# To use an adiabatic temperature profile, one needs to pin the temperature at the top of the lower mantle
T0 = 1900 #K
# To simplify we only compute temperatures for one composition
temperatures = burnman.geotherm.adiabatic(pressures, T0, pyrolitic_mantle)

In [ ]:
# plot temperature
plt.plot(pressures/1.e9,temperatures,'r')
plt.xlim(min(pressures)/1.e9,max(pressures)/1.e9)
plt.xlabel('pressure (GPa)')
plt.ylabel('temperature (K)')


#### 2.6 Calculate velocities

    
Here is the step which does the heavy lifting.  burnman.velocities_from_rock sets the state of the rock at each of the pressures and temperatures defined,then calculates the elastic moduli and density of each individual phase.  After that,it performs elastic averaging on the phases to get a single bulk and shear
modulus for the rock. Finally, it calculates the seismic
wave speeds for the whole rock.  It returns a tuple of density, p-wave velocity
s-wave velocity, bulk sound speed, bulk modulus, and shear modulus.

In [ ]:
print("Calculations are done for:")
pyrolitic_mantle.debug_print()
pyrolitic_vp, pyrolitic_vs, pyrolitic_rho = pyrolitic_mantle.evaluate(
        ['v_p', 'v_s', 'density'], pressures, temperatures)
print("Calculations are done for:")
chondritic_mantle.debug_print()
chondritic_vp, chondritic_vs, chondritic_rho = chondritic_mantle.evaluate(
        ['v_p', 'v_s', 'density'], pressures, temperatures)
print("Calculations are done for:")
your_mantle.debug_print()
yourmantle_vp, yourmantle_vs, yourmantle_rho = your_mantle.evaluate(
        ['v_p', 'v_s', 'density'], pressures, temperatures)

#### 2.7 Plot results

All the work is done except the plotting!  Here we want to plot the seismic wave
speeds and the density against PREM using the matplotlib plotting tools. 
First we plot the results with pressure and then convert to depths to plot with depth.

In [ ]:

# plot vp

ax[0].plot(depths / 1.e3, pyrolitic_vp / 1.e3, color='g', linestyle='-', marker='o',
             markerfacecolor='g', markersize=4, label='pyrolitic mantle')
ax[0].plot(depths / 1.e3, chondritic_vp / 1.e3, color='y', linestyle='-', marker='o',
             markerfacecolor='y', markersize=4, label='chondritic mantle')
ax[0].legend(loc='lower right')
ax[0].set_xlabel('depth(km)')
ax[0].set_title('P wave velocities ($km/s$)')

# plot Vs
ax[1].plot(depths / 1.e3, pyrolitic_vs / 1.e3, color='g', linestyle='-', marker='o',
             markerfacecolor='g', markersize=4)
ax[1].plot(depths / 1.e3, chondritic_vs / 1.e3, color='y', linestyle='-', marker='o',
             markerfacecolor='y', markersize=4)
ax[1].set_xlabel('depth(km)')
ax[1].set_title('S wave velocities ($km/s$)')

# plot density

ax[2].plot(depths / 1.e3, pyrolitic_rho / 1.e3, color='g', linestyle='-', marker='o',
             markerfacecolor='g', markersize=4)
ax[2].plot(depths / 1.e3, chondritic_rho / 1.e3, color='y', linestyle='-', marker='o',
             markerfacecolor='y', markersize=4)
ax[2].set_xlabel('depth(km)')
ax[2].set_title('densities ($g/cm^3$)')

'''
# plot your mantle
ax[0].plot(depths / 1.e3, yourmantle_vp / 1.e3, color='c', linestyle='-', marker='o',
                      markerfacecolor='c', markersize=4, label='your mantle')
plt.legend(loc='lower right')

ax[1].plot(depths / 1.e3, yourmantle_vs / 1.e3, color='c', linestyle='-', marker='o',
                      markerfacecolor='c', markersize=4)
           
ax[2].plot(depths / 1.e3,  yourmantle_rho / 1.e3, color='y', linestyle='-', marker='o',
            markerfacecolor='c', markersize=4)
'''
fig

### Step 3. Explore 3D velocities

Another main discussion in the lower mantle is if lateral velocity variations can be explained by temperature alone, or if there are also compositional variations. 

Running step 3 plots ~ 1000 equally spaced radial profiles from a 3D tomographic model (French & Romanowicz 2015) for shear velocity deviation. This gives a sense of the variation in shear velocity seen in the lower mantle (with respect to a 1D average). 

e.	How strong are the velocity variations across most of the lower mantle? How about at the bottom of the mantle?


f.	What reasons cause tomographic models to underestimate the amplitudes of the velocity variation? For these reasons we will look at the extreme deviations mapped for our interpretation.




In [ ]:
# Initialize plot
# load 3D seismic model into dictionary
# This model contains ~10000 equally spaced profiles of shear wave velocity for the
# model of French and Romanowicz(2015) filtered up to spherical harmonic degree 18.
seis3D =np.load('SEMUCBWM1_Lmax18.npy',encoding='latin1', allow_pickle=True).item()
figs = [plt.figure( figsize = (15,8)) for f in range(2)]
axes = [figs[f].add_subplot(1,1, 1) for f in range(2)]
for f in range(2):

    # plots every 10th profile in grey
    for prof in range(0,len(seis3D['lons']),10):
        axes[f].plot(seis3D['depths']/1.e3,seis3D['dVs'][:,prof],'k',alpha=0.02)

    # Plot min and max values in black
    axes[f].plot(seis3D['depths']/1.e3,np.max(seis3D['dVs'],axis=1),'k')
    axes[f].plot(seis3D['depths']/1.e3,np.min(seis3D['dVs'],axis=1),'k')
    axes[f].set_xlim([660.,2891.])
    axes[f].set_xlabel('depth ($km$)')
    axes[f].set_ylabel('dVs/Vs')

# This plot will show up twice for later purposes

### Step 4. Temperature vs. Compositional variations

Next we will explore the relative velocity variations caused by temperature versus compositional variations (in this case only iron content). A pyrolitic model (with 8% iron content) is used as the reference velocity). Run step 4 and look at the models as function of iron content and temperature. 
g.	What happens to the sensitivity of shear wave velocity to temperature and compositional variations with depth? Why might this be?



h.	How much temperature variation is needed to explain velocity variations at the top of the mantle? How about iron content? How about at the bottom of the mantle?




i.	If temperature variations explained the velocity variations, what density deviations would be associated with the fast and slow velocities? What if the velocity variations where due to iron content? To answer this question, you might want modify the code to plot density deviation instead of velocity deviation.



j. 	At the very base of the mantle, ultra-low velocity zones are observed using high-frequency body waves. These zones have shear wave velocity reductions up to 30%. Can iron enrichment alone explain such a velocity reduction? What other factors should/could play a role?  




#### 4.1 Compute velocities reference composition

In [ ]:
# Perovksite solid solution
frac_mg = 0.94
frac_fe = 0.06
frac_al = 0.00
mg_fe_perovskite = minerals.SLB_2011.mg_fe_perovskite()
mg_fe_perovskite.set_composition([frac_mg, frac_fe, frac_al])

# ferropericlase solid solution
frac_mg = 0.8
frac_fe = 0.2
mg_fe_periclase = minerals.SLB_2011.ferropericlase()
mg_fe_periclase.set_composition([frac_mg,frac_fe])

# Ca Perovsktie
ca_perovskite = minerals.SLB_2011.ca_perovskite()

# Pyrolitic composition
pyr_pv = 0.75
pyr_fp = 0.18
pyr_capv = 0.07
pyrolitic_mantle = burnman.Composite([mg_fe_perovskite, mg_fe_periclase, ca_perovskite], [pyr_pv, pyr_fp, pyr_capv])
    
# To use an adiabatic temperature profile, one needs to pin the temperature at the top of the lower mantle
T0 = 1900 #K
temperatures = burnman.geotherm.adiabatic(pressures, T0, pyrolitic_mantle)
 
print("Calculations are done for:")
pyrolitic_mantle .debug_print()
 
reference_vp, reference_vs, reference_rho = pyrolitic_mantle.evaluate(['v_p', 'v_s', 'density'], pressures, temperatures)

#### 4.2 Compute variation with temperature

In [ ]:

deltaT = np.arange(-350.,350.01,100.) # Range of temperature deviations to test np.arange(start, stop, step)
colorsT = [ cm.coolwarm(x) for x in np.linspace(0.,1.,len(deltaT)) ] # Color scale for resutls

 
for i, dT in enumerate(deltaT):
    T0 = 1900+dT # Deviate temperature
    temperatures = burnman.geotherm.adiabatic(pressures, T0, pyrolitic_mantle)
    
    print("Calculations are done for:", dT, " K")

    
    # Recompute velocities for new temperature
    mod_vp, mod_vs, mod_rho = pyrolitic_mantle.evaluate(['v_p', 'v_s', 'density'], pressures, temperatures)
    
    # Compute differential velocity to reference
    dlnVs = (mod_vs-reference_vs)/ reference_vs
    # Plot results on top of the 3D variations plot
    axes[0].plot(depths/1.e3, dlnVs, color=colorsT[i], linewidth =2, label = str(dT) + ' K')
axes[0].set_xlabel('depth (km)')
axes[0].set_ylabel('dVs/Vs')
axes[0].set_ylim([-0.04,0.04])

#Plot legend
handles, labels = axes[0].get_legend_handles_labels()
by_label = OrderedDict(zip(labels, handles))
# Put legend to the right of the plot
axes[0].legend(by_label.values(), by_label.keys(),loc='center left', bbox_to_anchor=(1, 0.5))
figs[0]

#### 4.2 Compute variation in iron content

In [ ]:
rangeFe = np.arange(0,0.31,.05)
colorsFe = [ cm.copper_r(x) for x in np.linspace(0.,1.,len(rangeFe)) ] # using the reversed copper scale

for i, dFe in enumerate(rangeFe):
    print(dFe)
    frac_pv =0.75
    frac_fp = 0.18
    # very simplified Fe partitioning between Perovskite and Ferropericlase. 
    dFepv = dFe
    dFefp = dFe
    print(dFe)
    # Perovksite solid solution
    frac_mg = 1.0 - dFepv
    frac_fe = dFepv
    frac_al = 0.00
    mg_fe_perovskite = minerals.SLB_2011.mg_fe_perovskite()
    mg_fe_perovskite.set_composition([frac_mg, frac_fe, frac_al])
    
    # ferropericlase solid solution
    frac_mg = 1.0 - dFefp
    frac_fe = dFefp
    mg_fe_periclase = minerals.SLB_2011.ferropericlase()
    mg_fe_periclase.set_composition([frac_mg,frac_fe])
    
    # Ca Perovskite
    ca_perovskite = minerals.SLB_2011.ca_perovskite()
    
    # Pyrolitic composition
    pyr_pv = frac_pv
    pyr_fp = frac_fp
    pyr_capv = 1.-frac_pv-frac_fp
    pyrolitic_mantle_fe = burnman.Composite([mg_fe_perovskite, mg_fe_periclase, ca_perovskite], [pyr_pv, pyr_fp, pyr_capv])
    T0 = 1900 #K
    temperatures = burnman.geotherm.adiabatic(pressures, T0, pyrolitic_mantle_fe)
    
    print("Calculations are done for iron fraction of ", dFe)

    
    mod_vp, mod_vs, mod_rho = pyrolitic_mantle_fe.evaluate(['v_p', 'v_s', 'density'], pressures, temperatures)

    # Compute differential velcoity to reference
    dlnVs = (mod_vs -reference_vs)/ reference_vs


    # Plot results
    axes[1].plot(depths/1.e3, dlnVs, color = colorsFe[i], linewidth =2, linestyle = '--', label = str(dFe) + ' % dFe ')
axes[1].set_xlabel('depth (km)')
axes[1].set_ylabel('dVs/Vs')
axes[1].set_ylim([-0.14,0.14])

#Plot legend
handles, labels = axes[1].get_legend_handles_labels()
by_label = OrderedDict(zip(labels, handles))
# Put legend to the right of the plot
axes[1].legend(by_label.values(), by_label.keys(),loc='center left', bbox_to_anchor=(1, 0.5))
figs[1]